In [2]:
# Import required libraries and dependencies
import pandas as pd
import numpy as np
from prophet import Prophet
import datetime as dt
import scipy.stats as sts
import hvplot.pandas

US crimes July 2016 - August 2022

In [3]:
# Read the CSV file into a `pandas` DataFrame - US Crimes Data Set
crime_df = pd.read_csv("resources/Crime.csv", low_memory=False)

# Update column names and set max columns
crime_df.columns = crime_df.columns.str.lower()
crime_df.columns = crime_df.columns.str.replace(' ', '_')
pd.set_option("display.max_columns", None)

crime_df.rename(columns={
    "dispatch_date_/_time": "dispatch_date_time"
}, inplace=True)

# Display df head and info
display(crime_df.head())
display(crime_df.info())

,incident_id,offence_code,cr_number,dispatch_date_time,nibrs_code,victims,crime_name1,crime_name2,crime_name3,police_district_name,block_address,city,state,zip_code,agency,place,sector,beat,pra,address_number,street_prefix,street_name,street_suffix,street_type,start_date_time,end_date_time,latitude,longitude,police_district_number,location
0,201202980,3550,180042096,08/23/2018 09:52:08 PM,35B,1,Crime Against Society,Drug Equipment Violations,DRUGS - NARCOTIC EQUIP - POSSESS,GERMANTOWN,12800 BLK MIDDLEBROOK RD,GERMANTOWN,MD,20874.0,MCPD,Street - In vehicle,N,5N1,447,12800.0,NaN,MIDDLEBROOK,NaN,RD,08/23/2018 09:52:00 PM,NaN,39.177744,-77.265619,5D,"(39.1777, -77.2656)"
1,201181293,3522,180015424,03/30/2018 01:00:55 AM,35A,1,Crime Against Society,Drug/Narcotic Violations,DRUGS - OPIUM OR DERIVATIVE - POSSESS,BETHESDA,8300 BLK WOODMONT AVE,BETHESDA,MD,20814.0,MCPD,Street - In vehicle,E,2E2,054,8300.0,NaN,WOODMONT,NaN,AVE,03/30/2018 01:01:00 AM,NaN,38.992693,-77.097063,2D,"(38.9927, -77.0971)"
2,201181293,3562,180015424,03/30/2018 01:00:55 AM,35A,1,Crime Against Society,Drug/Narcotic Violations,DRUGS - MARIJUANA - POSSESS,BETHESDA,8300 BLK WOODMONT AVE,BETHESDA,MD,20814.0,MCPD,Street - In vehicle,E,2E2,054,8300.0,NaN,WOODMONT,NaN,AVE,03/30/2018 01:01:00 AM,NaN,38.992693,-77.097063,2D,"(38.9927, -77.0971)"
3,201193163,3520,180029476,06/14/2018 10:26:45 PM,35A,1,Crime Against Society,Drug/Narcotic Violations,DRUGS - OPIUM OR DERIVATIVE - SELL,MONTGOMERY VILLAGE,400 BLK QUINCE ORCHARD RD,GAITHERSBURG,MD,20878.0,GPD,Hotel/Motel/Etc.,P,6P2,445,400.0,NaN,QUINCE ORCHARD,NaN,RD,06/14/2018 10:26:00 PM,06/15/2018 03:00:00 AM,39.147954,-77.218189,6D,"(39.148, -77.2182)"
4,201204355,2204,180043926,09/03/2018 12:06:54 PM,220,1,Crime Against Property,Burglary/Breaking and Entering,BURGLARY - NO FORCED ENTRY-RESIDENTIAL,BETHESDA,4800 BLK FALSTONE AVE,CHEVY CHASE,MD,20815.0,MCPD,Residence - Single Family,E,2E1,017,4800.0,NaN,FALSTONE,NaN,AVE,09/02/2018 11:30:00 PM,09/03/2018 12:30:00 AM,38.966174,-77.096561,2D,"(38.9662, -77.0966)"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306094 entries, 0 to 306093
Data columns (total 30 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   incident_id             306094 non-null  int64  
 1   offence_code            306094 non-null  object 
 2   cr_number               306094 non-null  int64  
 3   dispatch_date_time      257065 non-null  object 
 4   nibrs_code              306094 non-null  object 
 5   victims                 306094 non-null  int64  
 6   crime_name1             305822 non-null  object 
 7   crime_name2             305822 non-null  object 
 8   crime_name3             305822 non-null  object 
 9   police_district_name    306000 non-null  object 
 10  block_address           279888 non-null  object 
 11  city                    304818 non-null  object 
 12  state                   306094 non-null  object 
 13  zip_code                302915 non-null  float64
 14  agency              

None

US crime rates 1960-2014

In [4]:
# Read the CSV file into a `pandas` DataFrame - US Crime Rates 1960-2014 Data Set
us_crime_1960_2014_df = pd.read_csv("resources/US_Crime_Rates_1960_2014.csv", low_memory=False)

# Update column names and set max columns
us_crime_1960_2014_df.columns = us_crime_1960_2014_df.columns.str.lower()
us_crime_1960_2014_df.columns = us_crime_1960_2014_df.columns.str.replace(' ', '_')
pd.set_option("display.max_columns", None)

# Display df head and info
display(us_crime_1960_2014_df.head())
display(us_crime_1960_2014_df.info())

,year,population,total,violent,property,murder,forcible_rape,robbery,aggravated_assault,burglary,larceny_theft,vehicle_theft
0,1960,179323175,3384200,288460,3095700,9110,17190,107840,154320,912100,1855400,328200
1,1961,182992000,3488000,289390,3198600,8740,17220,106670,156760,949600,1913000,336000
2,1962,185771000,3752200,301510,3450700,8530,17550,110860,164570,994300,2089600,366800
3,1963,188483000,4109500,316970,3792500,8640,17650,116470,174210,1086400,2297800,408300
4,1964,191141000,4564600,364220,4200400,9360,21420,130390,203050,1213200,2514400,472800


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   year                55 non-null     int64
 1   population          55 non-null     int64
 2   total               55 non-null     int64
 3   violent             55 non-null     int64
 4   property            55 non-null     int64
 5   murder              55 non-null     int64
 6   forcible_rape       55 non-null     int64
 7   robbery             55 non-null     int64
 8   aggravated_assault  55 non-null     int64
 9   burglary            55 non-null     int64
 10  larceny_theft       55 non-null     int64
 11  vehicle_theft       55 non-null     int64
dtypes: int64(12)
memory usage: 5.3 KB


None

US juvenile arrests by crime 1995-2016

In [5]:
# Read the CSV file into a `pandas` DataFrame - US Juvenile Arrests
juvenile_arrests_df = pd.read_csv("resources/arrests_national_juvenile.csv", low_memory=False)

# Update column names and set max columns
juvenile_arrests_df.columns = juvenile_arrests_df.columns.str.lower()
juvenile_arrests_df.columns = juvenile_arrests_df.columns.str.replace(' ', '_')
pd.set_option("display.max_columns", None)

# Display df head and info
display(juvenile_arrests_df.head())
display(juvenile_arrests_df.info())

,id,year,state_abbr,offense_code,offense_name,agencies,population,total_male,total_female,m_0_9,m_10_12,m_13_14,m_15,m_16,m_17,f_0_9,f_10_12,f_13_14,f_15,f_16,f_17,race_agencies,race_population,white,black,asian_pacific_islander,american_indian
0,1081,2016,NaN,ASR_ARSON,Arson,13310,264534532,1760,328,117.0,368.0,548,291,239,197,12.0,54.0,114.0,66.0,39.0,43,12581,263887632,1436,516,41.0,48.0
1,1082,2016,NaN,ASR_AST,Aggravated Assault,13310,264534532,16997,5918,141.0,1406.0,3743,3062,3839,4806,19.0,465.0,1565.0,1171.0,1337.0,1361,12581,263887632,12370,9736,296.0,364.0
2,1083,2016,NaN,ASR_AST_SMP,Simple Assault,13310,264534532,66360,38712,1043.0,7406.0,16966,12403,14134,14408,259.0,3353.0,11210.0,8036.0,8264.0,7590,12581,263887632,59778,41923,1130.0,1378.0
3,1084,2016,NaN,ASR_BRG,Burglary,13310,264534532,23307,3071,167.0,1474.0,5301,4789,5662,5914,34.0,170.0,727.0,618.0,689.0,833,12581,263887632,14413,11082,368.0,362.0
4,1085,2016,NaN,ASR_CUR,Curfew and Loitering Law Violations,13310,264534532,19218,8319,110.0,1184.0,4160,4405,5126,4233,24.0,449.0,1991.0,1887.0,2267.0,1701,12581,263887632,15468,11045,382.0,429.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      690 non-null    int64  
 1   year                    690 non-null    int64  
 2   state_abbr              0 non-null      float64
 3   offense_code            690 non-null    object 
 4   offense_name            690 non-null    object 
 5   agencies                690 non-null    int64  
 6   population              690 non-null    int64  
 7   total_male              690 non-null    int64  
 8   total_female            690 non-null    int64  
 9   m_0_9                   654 non-null    float64
 10  m_10_12                 685 non-null    float64
 11  m_13_14                 690 non-null    int64  
 12  m_15                    690 non-null    int64  
 13  m_16                    690 non-null    int64  
 14  m_17                    690 non-null    in

None

Inflation rates by country 1970-2022

In [7]:
# Read the CSV file into a `pandas` DataFrame - Global Inflation Dataset (1970-2022)
inflation_df = pd.read_csv("resources/Global Dataset of Inflation.csv", encoding="ISO-8859-1",  low_memory=False)

# Update column names and set max columns
inflation_df.columns = inflation_df.columns.str.lower()
inflation_df.columns = inflation_df.columns.str.replace(' ', '_')
pd.set_option("display.max_columns", None)

# Display df head and info
display(inflation_df.head())
display(inflation_df.info())

,country_code,imf_country_code,country,indicator_type,series_name,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,note,unnamed:_59,unnamed:_60,unnamed:_61,unnamed:_62,unnamed:_63
0,ABW,314.0,Aruba,Inflation,Headline Consumer Price Inflation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.20,3.00,1.88,2.28,4.05,2.89,3.32,3.66,2.53,3.40,3.61,5.39,8.96,-2.14,2.08,4.32,0.63,-2.37,0.42,0.48,-0.89,-0.47,3.58,4.26,1.22,0.74,6.04,Annual average inflation,NaN,NaN,NaN,NaN,NaN
1,AFG,512.0,Afghanistan,Inflation,Headline Consumer Price Inflation,25.51,25.51,-12.52,-10.68,10.23,11.85,0.60,7.65,5.00,9.90,13.40,22.20,18.20,15.90,20.40,8.70,-2.10,18.40,27.50,71.50,47.40,43.80,58.19,33.99,20.01,14.00,14.01,14.01,14.01,14.01,0.00,-43.40,51.93,35.66,16.36,10.57,6.78,8.68,26.42,-6.81,2.18,11.80,6.44,7.39,4.67,-0.66,4.38,4.98,0.63,2.30,5.44,5.06,NaN,Annual average inflation,NaN,NaN,NaN,NaN,NaN
2,AGO,614.0,Angola,Inflation,Headline Consumer Price Inflation,7.97,5.78,15.80,15.67,27.42,29.00,80.70,69.01,48.46,101.30,46.71,1.39,1.83,1.83,1.83,1.83,1.83,1.83,1.83,1.83,1.83,85.26,299.10,1379.48,949.77,2672.23,4146.01,221.49,107.43,248.25,325.03,152.59,108.89,98.22,43.53,22.96,13.31,12.25,12.47,13.72,14.48,13.48,10.28,8.78,7.30,9.16,32.38,29.84,19.63,17.08,21.02,23.85,21.35,Annual average inflation,NaN,NaN,NaN,NaN,NaN
3,ALB,914.0,Albania,Inflation,Headline Consumer Price Inflation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.18,35.72,226.00,85.00,22.57,7.75,12.80,33.21,20.55,0.40,0.03,3.10,5.18,2.38,2.83,2.36,2.41,2.91,3.41,2.29,3.57,3.41,2.03,1.93,1.62,1.91,1.29,1.99,2.03,1.41,1.62,2.04,6.73,Annual average inflation,NaN,NaN,NaN,NaN,NaN
4,ARE,466.0,United Arab Emirates,Inflation,Headline Consumer Price Inflation,21.98,21.98,21.98,21.98,21.98,21.98,21.98,21.97,13.04,10.99,10.07,7.95,7.08,1.30,2.43,3.50,5.43,5.50,4.99,2.79,0.60,3.40,6.38,5.27,5.70,4.33,2.98,2.97,1.99,2.10,1.35,2.80,2.92,3.12,5.04,6.20,9.28,11.13,12.27,1.57,0.88,0.88,0.66,1.09,2.34,4.07,1.62,1.97,3.06,-1.93,-2.08,0.18,5.22,Annual average inflation,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783 entries, 0 to 782
Data columns (total 64 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   country_code      783 non-null    object 
 1   imf_country_code  781 non-null    float64
 2   country           783 non-null    object 
 3   indicator_type    783 non-null    object 
 4   series_name       783 non-null    object 
 5   1970              422 non-null    float64
 6   1971              428 non-null    float64
 7   1972              430 non-null    float64
 8   1973              430 non-null    float64
 9   1974              434 non-null    float64
 10  1975              434 non-null    float64
 11  1976              430 non-null    float64
 12  1977              427 non-null    float64
 13  1978              428 non-null    float64
 14  1979              428 non-null    float64
 15  1980              433 non-null    float64
 16  1981              451 non-null    float64
 1

None

In [8]:
# Sort values and display tail
inflation_df.sort_values("country", inplace=True)
inflation_df.tail(50)

,country_code,imf_country_code,country,indicator_type,series_name,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,note,unnamed:_59,unnamed:_60,unnamed:_61,unnamed:_62,unnamed:_63
540,UGA,746.0,Uganda,Inflation,Food Consumer Price Inflation,13.200000,24.70,-5.50,18.60,75.50,23.50,62.00,79.50,NaN,NaN,NaN,NaN,NaN,38.50,41.80,174.80,128.40,197.00,197.10,NaN,19.20,23.90,65.30,-4.00,15.90,4.60,6.60,18.10,-2.00,6.90,1.00,-0.66,3.24,-0.21,-3.07,-6.87,5.94,2.66,19.85,25.25,2.66,29.26,13.61,2.25,3.82,7.43,5.53,10.10,-1.19,-0.20,1.76,0.67,12.64,Food and non-acoholic beverage,NaN,NaN,NaN,NaN,NaN
541,UKR,926.0,Ukraine,Inflation,Food Consumer Price Inflation,0.000000,0.40,0.20,0.10,0.50,0.10,-0.10,0.00,1.10,0.60,0.90,1.20,2.90,1.00,-0.20,1.80,4.70,4.70,-0.10,0.60,NaN,NaN,NaN,5414.10,799.20,355.90,58.30,11.60,11.80,27.80,34.50,14.40,0.00,6.20,11.30,16.60,5.31,9.64,35.68,11.94,10.88,6.42,-2.13,-2.16,12.05,45.92,8.96,12.88,11.15,8.00,2.74,10.79,25.26,Food and non-acoholic beverage,NaN,NaN,NaN,NaN,NaN
180,UKR,926.0,Ukraine,Inflation,Headline Consumer Price Inflation,0.200000,-0.20,-0.10,0.00,0.10,0.10,0.00,0.40,0.80,1.10,1.30,1.00,2.80,0.70,-0.90,0.30,1.50,1.50,-0.10,2.20,2.20,2.20,2.20,4660.58,861.05,376.02,77.82,15.62,10.66,22.83,28.26,11.91,0.72,5.22,9.03,13.53,9.09,12.89,25.20,15.86,9.33,7.95,0.59,-0.23,12.15,48.57,13.95,14.44,10.94,7.89,2.73,9.36,20.18,Annual average inflation,NaN,NaN,NaN,NaN,NaN
361,UKR,926.0,Ukraine,Inflation,Energy Consumer Price Inflation,35.050000,35.05,35.05,35.05,89.00,73.40,61.40,54.70,32.00,55.50,60.80,17.60,19.30,74.70,62.60,72.20,60.60,65.06,67.15,82.75,105.85,101.10,73.77,61.73,55.82,45.39,33.33,22.88,12.55,6.48,4.57,4.07,11.41,20.79,9.56,6.14,8.13,8.86,11.16,10.70,8.85,10.32,9.24,10.70,9.03,8.11,9.20,8.58,6.06,5.91,-2.98,23.16,5.01,"Housing, Water, Electricity, Gas & Other Fuel",NaN,NaN,NaN,NaN,NaN
774,UKR,926.0,Ukraine,Inflation,Producer Price Inflation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14400.00,1140.00,488.10,51.83,7.67,13.20,31.13,20.86,8.65,3.05,7.71,20.32,16.71,9.49,19.48,35.56,6.59,20.88,19.00,3.65,-0.05,17.17,35.69,20.92,26.45,17.58,4.14,-1.88,40.82,61.05,"Producer Price Index, All Commodities",NaN,NaN,NaN,NaN,NaN
663,UKR,926.0,Ukraine,Inflation,Official Core Consumer Price Inflation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.41,8.59,7.70,3.32,0.23,9.86,40.39,11.00,7.33,9.09,6.77,3.30,6.87,14.62,All Items Excluding Food and Energy,NaN,NaN,NaN,NaN,NaN
379,ARE,466.0,United Arab Emirates,Inflation,Food Consumer Price Inflation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.50,0.79,4.45,5.86,5.24,2.69,2.17,1.24,1.01,1.20,3.54,-1.39,3.70,0.00,NaN,Food and non-acoholic beverage,NaN,NaN,NaN,NaN,NaN
206,ARE,466.0,United Arab Emirates,Inflation,Energy Consumer Price Inflation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.17,0.45,-0.30,-2.42,-2.58,0.25,3.12,8.81,3.56,0.87,-3.09,-5.20,-3.67,-3.65,NaN,Energy,NaN,NaN,NaN,NaN,NaN
4,ARE,466.0,United Arab Emirates,Inflation,Headline Consumer Price Inflation,21.980000,21.98,21.98,21.98,21.98,21.98,21.98,21.97,13.04,10.99,10.07,7.95,7.08,1.30,2.43,3.50,5.43,5.50,4.99,2.79,0.60,3.40,6.38,5.27,5.70,4.33,2.98,2.97,1.99,2.10,1.35,2.80,2.92,3.12,5.04,6.20,9.28,11.13,12.27,1.57,0.88,0.88,0.66,1.09,2.34,4.07,1.62,1.97,3.06,-1.93,-2.08,0.18,5.22,Annual average inflation,NaN,NaN,NaN,NaN,NaN
62,GBR,112.0,United Kingdom,Inflation,Headline Consumer Price Inflation,6.370000,9.44,7.07,9.20,

In [9]:
# Create US inflation df and display
inflation_us = inflation_df.loc[inflation_df["country_code"] == "USA"]
inflation_us

,country_code,imf_country_code,country,indicator_type,series_name,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,note,unnamed:_59,unnamed:_60,unnamed:_61,unnamed:_62,unnamed:_63
665,USA,111.0,United States,Inflation,Official Core Consumer Price Inflation,6.245934,4.69,3.06,3.48,8.24,9.22,6.57,6.25,7.31,9.74,12.42,10.47,7.40,3.93,4.94,4.38,4.04,4.12,4.41,4.50,5.03,4.91,3.66,3.30,2.85,2.99,2.70,2.39,2.29,2.08,2.43,2.67,2.32,1.46,1.76,2.17,2.50,2.34,2.30,1.70,0.96,1.66,2.11,1.76,1.75,1.83,2.21,1.84,2.14,2.19,1.70,3.57,6.22,All Items Excluding Food and Energy,NaN,NaN,NaN,NaN,NaN
776,USA,111.0,United States,Inflation,Producer Price Inflation,3.610000,3.33,4.41,13.14,18.81,9.25,4.65,6.13,7.77,12.54,14.13,9.13,2.01,1.26,2.38,-0.43,-2.89,2.64,4.02,4.96,3.56,0.22,0.59,1.46,1.30,3.58,2.34,-0.07,-2.48,0.84,5.78,1.11,-2.29,5.35,6.18,7.32,4.67,4.80,9.81,-8.84,4.48,3.87,1.86,1.36,1.58,-0.87,0.42,2.33,2.90,1.69,-2.12,8.95,7.91,"Producer Price Index, All Commodities",NaN,NaN,NaN,NaN,NaN
543,USA,111.0,United States,Inflation,Food Consumer Price Inflation,5.400000,3.07,4.13,13.29,13.76,8.44,3.09,5.96,9.77,10.71,8.56,7.77,4.07,2.23,3.76,2.33,3.26,4.07,4.13,5.67,5.76,3.56,1.38,2.09,2.32,2.76,3.25,2.62,2.12,2.15,2.31,3.09,1.84,2.14,3.37,2.43,2.35,3.91,5.37,1.88,0.79,3.58,2.55,1.41,2.31,1.80,0.34,0.88,1.40,0.89,3.51,3.47,11.35,Food and non-acoholic beverage,NaN,NaN,NaN,NaN,NaN
182,USA,111.0,United States,Inflation,Headline Consumer Price Inflation,5.900000,4.29,3.27,6.18,11.05,9.14,5.74,6.50,7.63,11.25,13.55,10.33,6.13,3.21,4.30,3.55,1.90,3.66,4.08,4.83,5.40,4.23,3.03,2.95,2.61,2.81,2.93,2.34,1.55,2.19,3.38,2.83,1.59,2.27,2.68,3.39,3.23,2.85,3.84,-0.36,1.64,3.16,2.07,1.46,1.62,0.12,1.26,2.14,2.44,1.81,1.23,4.70,8.00,Annual average inflation,NaN,NaN,NaN,NaN,NaN
363,USA,111.0,United States,Inflation,Energy Consumer Price Inflation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.97,2.33,16.90,7.10,25.60,21.27,2.59,21.01,27.12,"Housing, Water, Electricity, Gas & Other Fuel",NaN,NaN,NaN,NaN,NaN


Unenployment rate by country 1991-2021

In [10]:
# Read the CSV file into a `pandas` DataFrame - Unemployment
unemployment_df = pd.read_csv("resources/unemployment analysis.csv", low_memory=False)

# Update column names and set max columns
unemployment_df.columns = unemployment_df.columns.str.lower()
unemployment_df.columns = unemployment_df.columns.str.replace(' ', '_')
pd.set_option("display.max_columns", None)

# Display df head and info
display(unemployment_df.head())
display(unemployment_df.info())

,country_name,country_code,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Africa Eastern and Southern,AFE,7.80,7.84,7.85,7.84,7.83,7.84,7.86,7.81,7.79,7.72,7.73,7.96,7.79,7.31,7.12,6.99,6.74,6.27,6.32,6.87,6.75,6.56,6.45,6.41,6.49,6.61,6.71,6.73,6.91,7.56,8.11
1,Afghanistan,AFG,10.65,10.82,10.72,10.73,11.18,10.96,10.78,10.80,10.81,10.81,10.81,11.26,11.14,10.99,11.22,11.10,11.30,11.09,11.31,11.35,11.05,11.34,11.19,11.14,11.13,11.16,11.18,11.15,11.22,11.71,13.28
2,Africa Western and Central,AFW,4.42,4.53,4.55,4.54,4.53,4.57,4.60,4.66,4.86,4.92,4.87,4.78,4.75,4.73,4.73,4.65,4.63,4.60,4.58,4.55,4.55,4.64,4.41,4.69,4.63,5.57,6.02,6.04,6.06,6.77,6.84
3,Angola,AGO,4.21,4.21,4.23,4.16,4.11,4.10,4.09,4.07,4.05,4.03,4.00,3.96,3.96,3.92,3.88,3.86,3.82,3.79,3.78,9.43,7.36,7.35,7.37,7.37,7.39,7.41,7.41,7.42,7.42,8.33,8.53
4,Albania,ALB,10.31,30.01,25.26,20.84,14.61,13.93,16.88,20.05,20.84,19.03,18.58,17.90,16.99,16.31,15.97,15.63,15.97,13.06,13.67,14.09,13.48,13.38,15.87,18.05,17.19,15.42,13.62,12.30,11.47,13.33,11.82


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_name  235 non-null    object 
 1   country_code  235 non-null    object 
 2   1991          235 non-null    float64
 3   1992          235 non-null    float64
 4   1993          235 non-null    float64
 5   1994          235 non-null    float64
 6   1995          235 non-null    float64
 7   1996          235 non-null    float64
 8   1997          235 non-null    float64
 9   1998          235 non-null    float64
 10  1999          235 non-null    float64
 11  2000          235 non-null    float64
 12  2001          235 non-null    float64
 13  2002          235 non-null    float64
 14  2003          235 non-null    float64
 15  2004          235 non-null    float64
 16  2005          235 non-null    float64
 17  2006          235 non-null    float64
 18  2007          235 non-null    

None

In [11]:
# Sort values and display tail
unemployment_df.sort_values("country_name", inplace=True)
unemployment_df.tail(20)

,country_name,country_code,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
215,Tunisia,TUN,15.65,15.58,15.83,15.86,15.94,15.83,15.94,15.61,15.21,14.94,14.40,14.55,14.51,14.23,12.87,12.51,12.36,12.44,13.29,13.05,18.33,17.63,15.93,15.06,15.16,15.56,15.33,15.46,15.13,16.59,16.82
216,Turkiye,TUR,8.21,8.51,8.96,8.58,7.64,6.63,6.84,6.89,7.69,6.50,8.38,10.36,10.54,10.84,10.64,8.72,8.87,9.71,12.55,10.66,8.80,8.15,8.73,9.88,10.24,10.84,10.82,10.89,13.67,13.11,13.39
207,Turkmenistan,TKM,1.40,1.50,1.50,4.70,8.30,10.30,11.00,12.20,12.70,11.54,10.68,9.75,8.99,8.19,7.50,6.61,5.76,4.95,4.00,4.00,4.09,4.09,4.11,4.14,4.14,4.17,4.20,4.23,4.27,4.95,5.08
218,Uganda,UGA,0.94,0.92,1.16,1.42,1.65,1.92,2.21,2.47,2.71,3.03,3.27,3.50,3.60,2.75,1.90,2.29,2.73,3.15,3.60,3.59,3.51,3.55,1.91,1.91,1.91,1.92,1.93,1.92,1.92,2.77,2.94
219,Ukraine,UKR,1.90,1.90,2.00,2.00,5.62,7.65,8.93,11.32,11.86,11.71,11.06,10.14,9.06,8.59,7.18,6.81,6.35,6.36,8.84,8.10,7.85,7.53,7.17,9.27,9.14,9.35,9.50,8.80,8.19,9.13,8.88
6,United Arab Emirates,ARE,1.87,1.84,1.85,1.81,1.80,1.90,1.98,2.14,2.22,2.25,2.49,2.65,2.75,2.91,3.12,2.94,2.87,2.74,2.68,2.48,2.30,2.18,2.04,1.91,1.77,1.64,2.46,2.35,2.23,3.19,3.36
71,United Kingdom,GBR,8.55,9.78,10.35,9.65,8.69,8.19,7.07,6.20,6.04,5.56,4.70,5.04,4.81,4.59,4.75,5.35,5.26,5.62,7.54,7.79,8.04,7.88,7.52,6.11,5.30,4.81,4.33,4.00,3.74,4.47,4.53
222,United States,USA,6.80,7.50,6.90,6.12,5.65,5.45,5.00,4.51,4.22,3.99,4.73,5.78,5.99,5.53,5.08,4.62,4.62,5.78,9.25,9.63,8.95,8.07,7.37,6.17,5.28,4.87,4.36,3.90,3.67,8.05,5.46
220,Upper middle income,UMC,3.82,3.87,4.19,4.62,5.01,5.18,5.34,5.61,5.76,5.52,5.77,6.05,6.22,6.01,5.93,5.70,5.45,5.49,5.99,5.67,5.54,5.48,5.49,5.49,5.66,5.90,5.93,5.77,6.00,6.76,6.76
221,Uruguay,URY,8.95,8.96,8.35,9.04,10.02,11.93,11.44,9.36,10.47,12.63,15.05,16.65,16.66,12.98,12.01,10.84,9.40,8.03,7.74,7.16,6.31,6.45,6.44,6.55,7.49,7.84,7.89,8.34,8.88,10.35,10.45


In [12]:
# Create US unemployment df and display
unemployment_us = unemployment_df.loc[unemployment_df["country_name"] == "United States"]
unemployment_us

,country_name,country_code,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
222,United States,USA,6.8,7.5,6.9,6.12,5.65,5.45,5.0,4.51,4.22,3.99,4.73,5.78,5.99,5.53,5.08,4.62,4.62,5.78,9.25,9.63,8.95,8.07,7.37,6.17,5.28,4.87,4.36,3.9,3.67,8.05,5.46
